In [0]:
# importation des bibliotheques :
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [0]:
# chargement de donnees :
df = pd.read_csv('/Volumes/workspace/trips/echantillon/yellowtaxisample1pct_hybrid_stratified.csv')
df.tail()

# **Étape 1 : Analyse exploratoire (EDA)**


## 1️⃣ Structure initiale de l'echantillon


In [0]:
# taille de donnees
shape_data = df.shape

print(f'Le jeu de donnees contient {shape_data[0]} lignes et {shape_data[1]} colonnes.')

In [0]:
# colonnes
nom_colonnes = df.columns

print(f'Les noms des colonnes sont :\n{nom_colonnes.tolist()}')
# Rq : deux colonnes a merger => 'airport_fee', 'Airport_fee'

### 🚩 Remarque:

Les deux colonnes 'airport_fee' et 'Airport_fee' représentent la même information, mais réparties différemment
- Les valeurs non nulles apparaissent toujours dans l’une ou l’autre mais jamais en même temps
Donc : on va fusionner en une seule colonne propre appele airport_fee_merged

In [0]:
# types de donnees
types_donnees = df.info()
print(f"Les types de donnees sont :\n{'-'* 28}\n{types_donnees}")

# Rq: tpep_pickup_datetime et tpep_dropoff_datetime doivent etre de type datetime

### Compréhension fonctionnelle des variables

#### ⏰ Variables temporelles
- `tpep_pickup_datetime`
- `tpep_dropoff_datetime`

---

#### 💰 Variables liées au prix
- `fare_amount`
- `total_amount`
- `extra`
- `mta_tax`
- `tolls_amount`
- `improvement_surcharge`
- `congestion_surcharge`
- `airport_fee`

---

#### 🌍 Variables géographiques
- `PULocationID`
- `DOLocationID`

---

#### 👥 Variables liées au comportement client
- `passenger_count`
- `trip_distance`
- `payment_type`
- `tip_amount`

---

#### 🧾 Variables techniques
- `VendorID`
- `store_and_fwd_flag`


In [0]:
# convertion de type des colonnes temporelles

# Comptage des valeurs NaT avant conversion
print("Valeurs NaT avant conversion :")
print(f"tpep_pickup_datetime: {df['tpep_pickup_datetime'].isna().sum()}")
print(f"tpep_dropoff_datetime: {df['tpep_dropoff_datetime'].isna().sum()}")
print('*'*30)

# Format mixed (le plus courant dans ce type de dataset)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='mixed')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='mixed')

# Comptage des valeurs NaT après conversion
print("Valeurs NaT après conversion :")
print(f"tpep_pickup_datetime: {df['tpep_pickup_datetime'].isna().sum()}")
print(f"tpep_dropoff_datetime: {df['tpep_dropoff_datetime'].isna().sum()}")
print('*'*30)

# verifier le type:
df[['tpep_pickup_datetime','tpep_dropoff_datetime']].dtypes


## 2️⃣ Analyse de qualite de donnees

In [0]:
# merger airport_fee et Airport_fee
df['airport_fee_merged'] = df['airport_fee'].fillna(df['Airport_fee'])

# suppression de la colonne Airport_fee
df.drop(columns=['Airport_fee'], inplace=True)
df.drop(columns=['airport_fee'], inplace=True)

In [0]:
# valeurs manquantes par colonne
valeurs_nulles = df.isnull().sum()
taux_valeurs_nulles = round((df.isnull().sum() / len(df)) * 100, 2)

df_nulles = pd.DataFrame({
    'Colonne': valeurs_nulles.index,
    'Nb_Manquants': valeurs_nulles.values,
    'Pourcentage': taux_valeurs_nulles.values
}).sort_values('Nb_Manquants', ascending=False)

print(df_nulles[df_nulles['Nb_Manquants'] > 0])

In [0]:
# valeurs dupliquées dans l'echantillonage
nb_duplicats = df.duplicated().sum()
print(f'Nombre de lignes dupliquées : {nb_duplicats}')

In [0]:
# valeur dupliquées dans le cas ou les colonnes airport_fee_merged, congestion_surcharge, RatecodeID, passenger_count, store_and_fwd_flag  ont des valeurs manquant

nb_duplicats = df[df['airport_fee_merged'].isna()][['airport_fee_merged', 'congestion_surcharge', 'RatecodeID', 'passenger_count', 'store_and_fwd_flag']].duplicated().sum()
print(f'Nombre de lignes dupliquées : {nb_duplicats}')

### 🚩 Remarque: 
- Les valeurs nulles dans plusieurs colonnes apparaissent sur les mêmes lignes, et concernent uniquement :

`airport_fee_merged`,
 `congestion_surcharge`,
 `RatecodeID`,
 `passenger_count`,
 `store_and_fwd_flag`

## 3️⃣ Analyse des Incohérences

In [0]:
# duree de la course : (en minutes)
df["trip_duration_min"] = (
    df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
).dt.total_seconds() / 60

# duree de course anormale
out_duration_negatif = df[(df["trip_duration_min"] < 0)]
out_duration_null = df[(df["trip_duration_min"] == 0)]


len(out_duration_negatif), len(out_duration_null)


In [0]:
# les dates sont cohérentes?
out_dates_drop_avant_pickup = df[
    (df["tpep_dropoff_datetime"] < df["tpep_pickup_datetime"])
]

out_dates_drop_pickup_egaux = df[
    (df["tpep_dropoff_datetime"] == df["tpep_pickup_datetime"])
]

len(out_dates_drop_avant_pickup), len(out_dates_drop_pickup_egaux)

In [0]:
# distance anormale
out_trip_distance_negatif = df[
    df["trip_distance"] < 0
]

out_trip_distance_null = df[
    df["trip_distance"] == 0
]

len(out_trip_distance_negatif), len(out_trip_distance_null)

In [0]:
# nombres de passagers
value_counts = df['passenger_count'].value_counts(dropna=False)
value_percent = round(df['passenger_count'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
print(len(df[df['passenger_count'] >= 7]))

In [0]:
# type de tarif
value_counts = df['RatecodeID'].value_counts(dropna=False)
value_percent = round(df['RatecodeID'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
print(len(df[df['RatecodeID'] == 99]))

In [0]:
# fare amount anormal
out_fare_amount_negatif = df[(df["fare_amount"] < 0)]
out_fare_amount_null = df[(df["fare_amount"] == 0)]


print(df['fare_amount'].quantile(0.99),'/', len(df[df['fare_amount'] > df['fare_amount'].quantile(0.99)]))
print('*'*70)
len(out_fare_amount_negatif), len(out_fare_amount_null)

In [0]:
# mta_tax anormal 
value_counts = df['mta_tax'].value_counts(dropna=False)
value_percent = round(df['mta_tax'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
out_mta_tax_negatif = df[(df["mta_tax"] < 0)]
print(df['mta_tax'].quantile(0.99),'/', len(df[df['mta_tax'] > df['mta_tax'].quantile(0.99)]))
print('*'*70)
len(out_mta_tax_negatif)

In [0]:
# tip_amount anormal
out_tip_amount_negatif = df[(df["tip_amount"] < 0)]

print(df['tip_amount'].quantile(0.99), len(df[df['tip_amount'] > df['tip_amount'].quantile(0.99)]))
print('*'*70)
len(out_tip_amount_negatif)


In [0]:
# tolls amount anormal
out_tolls_amount_negatif = df[(df["tolls_amount"] < 0)]

print(df['tolls_amount'].quantile(0.99),'/', len(df[df['tolls_amount'] > df['tolls_amount'].quantile(0.99)]))
print('*'*70)
len(out_tolls_amount_negatif)

In [0]:
# improvement surcharge anormal
value_counts = df['improvement_surcharge'].value_counts(dropna=False)
value_percent = round(df['improvement_surcharge'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
out_improvement_surcharge_negatif = df[(df["improvement_surcharge"] < 0)]
print(df['improvement_surcharge'].quantile(0.99),'/', len(df[df['improvement_surcharge'] >  df['improvement_surcharge'].quantile(0.99)]))
print('*'*70)
len(out_improvement_surcharge_negatif) 

In [0]:
# total_amount anormal
out_total_amount_negatif = df[(df["total_amount"] < 0)]
out_total_amount_null = df[(df["total_amount"] == 0)]

total_amount_sum = df[df['total_amount'] != df['fare_amount'] + df['extra'] + df['mta_tax'] + df['tip_amount'] + df['tolls_amount'] + df['improvement_surcharge'] + df['congestion_surcharge']]

print(df['tolls_amount'].quantile(0.99),'/', len(df[df['total_amount'] > df['total_amount'].quantile(0.99)]))
print('*'*70)
len(out_tolls_amount_negatif), len(out_total_amount_null), len(total_amount_sum)



In [0]:
# congestion_surcharge anormal :
value_counts = df['congestion_surcharge'].value_counts(dropna=False)
value_percent = round(df['congestion_surcharge'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
out_congestion_surcharge_negatif = df[(df["congestion_surcharge"] < 0)]
print(df['congestion_surcharge'].quantile(0.99),'/', len(df[df['congestion_surcharge'] >  df['congestion_surcharge'].quantile(0.99)]))
print('*'*70)
len(out_congestion_surcharge_negatif) 

In [0]:
# airport_fee_merged anormal :
value_counts = df['airport_fee_merged'].value_counts(dropna=False)
value_percent = round(df['airport_fee_merged'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
out_airport_fee_merged_negatif = df[(df["airport_fee_merged"] < 0)]
print(df['airport_fee_merged'].quantile(0.99),'/', len(df[df['airport_fee_merged'] >  df['airport_fee_merged'].quantile(0.99)]))
print('*'*70)
len(out_airport_fee_merged_negatif) 

In [0]:
# incoherent combinees 
out_combined = {
    "distance_zero_fare_pos": len(df[(df["trip_distance"] == 0) & (df["fare_amount"] > 0)]),
    "distance_zero_duration_pos": len(df[(df["trip_distance"] == 0) & (df["trip_duration_min"] > 0)]),
    "tip_superieur_fare": len(df[df["tip_amount"] > df["fare_amount"]])
}

out_combined


##4️⃣ Distribution des donnees 

In [0]:
cols = [
    "RatecodeID",
    "payment_type",
    "store_and_fwd_flag",
    "passenger_count"
]

for col in cols:
    print(f"\n📌 Distribution de la colonne : {col}")

    # Comptage
    value_counts = df[col].value_counts(dropna=False)
    value_percent = round(df[col].value_counts(normalize=True, dropna=False) * 100, 2)

    result = pd.DataFrame({
        "Valeur": value_counts.index,
        "Nombre": value_counts.values,
        "Pourcentage (%)": value_percent.values
    })

    print(result)

    # Visualisation
    plt.figure(figsize=(10, 4))
    value_counts.plot(kind="barh")
    plt.title(f"Distribution de {col}")
    plt.xlabel("Nombre d'occurrences")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# **Étape 2 : Nettoyage des donnees**


In [0]:
df_clean = df.copy()

##1️⃣ les valeurs incoherants  

In [0]:
# -------------------------------
# GESTION DES OUTLIERS - ÉCHANTILLON
# -------------------------------

# --- Durée de la course ---
# On supprime uniquement les durées négatives (impossibles)
df_clean = df[df["trip_duration_min"] > 0].copy()

# Flag pour les durées extrêmes (99e percentile)
duration_99 = df_clean["trip_duration_min"].quantile(0.99)
df_clean["flag_duration_extreme"] = df_clean["trip_duration_min"] > duration_99

# --- Distance de la course ---
# Flag distance zéro (potentiellement incohérente si fare > 0)
df_clean["flag_distance_zero"] = df_clean["trip_distance"] == 0

# Suppression des distances zéro incohérentes avec un tarif positif
df_clean = df_clean[~((df_clean["trip_distance"] == 0) & (df_clean["fare_amount"] > 0))]

# --- Fare amount ---
# Suppression des valeurs négatives
df_clean = df_clean[df_clean["fare_amount"] >= 0]

# Flag pour les fares extrêmes (> 99e percentile)
fare_99 = df_clean["fare_amount"].quantile(0.99)
df_clean["flag_fare_extreme"] = df_clean["fare_amount"] > fare_99

# Winsorisation (optionnel pour analyse inférentielle)
df_clean["fare_amount"] = df_clean["fare_amount"].clip(upper=fare_99)

# --- Tip amount ---
# Suppression des valeurs négatives
df_clean = df_clean[df_clean["tip_amount"] >= 0]

# Correction tips supérieurs au montant de la course
df_clean.loc[df_clean["tip_amount"] > df_clean["fare_amount"], "tip_amount"] = df_clean["fare_amount"]

# Flag pour tips extrêmes (> 99e percentile)
tip_99 = df_clean["tip_amount"].quantile(0.99)
df_clean["flag_tip_extreme"] = df_clean["tip_amount"] > tip_99
df_clean["tip_amount"] = df_clean["tip_amount"].clip(upper=tip_99)

# --- Total amount ---
# Suppression des valeurs négatives
df_clean = df_clean[df_clean["total_amount"] >= 0]

# Recalcul du total (hors outliers, juste pour vérification)
df_clean["total_recalculated"] = (
    df_clean["fare_amount"] + df_clean["extra"] + df_clean["mta_tax"] +
    df_clean["tip_amount"] + df_clean["tolls_amount"] +
    df_clean["improvement_surcharge"] + df_clean["congestion_surcharge"]
)

# On peut filtrer si différence trop grande (>0.5$)
df_clean = df_clean[abs(df_clean["total_amount"] - df_clean["total_recalculated"]) < 0.50]

# --- Montants annexes ---
for col in ["mta_tax", "tolls_amount", "improvement_surcharge", "congestion_surcharge", "airport_fee_merged"]:
    # Suppression des valeurs négatives
    df_clean = df_clean[df_clean[col] >= 0]
    # Flag pour outliers (> 99e percentile)
    q99 = df_clean[col].quantile(0.99)
    df_clean[f"flag_{col}_extreme"] = df_clean[col] > q99
    # Winsorisation pour comparaison
    df_clean[col] = df_clean[col].clip(upper=q99)

# --- Passenger count ---
# Flag anomalies rares (0,7,8,9) sans toucher aux NaN
df_clean["flag_passenger_anomaly"] = df_clean["passenger_count"].isin([0, 7, 8, 9])

# --- RatecodeID ---
# Flag pour RatecodeID rare ou incohérent (ex: 99)
df_clean["flag_ratecode_rare"] = df_clean["RatecodeID"] == 99

# --- Résumé ---
print(f"Taille initiale : {len(df)}")
print(f"Taille après suppression des outliers : {len(df_clean)}")


##2️⃣ les valeurs manquantes

In [0]:
# -------------------------------
# NETTOYAGE DES VALEURS MANQUANTES
# -------------------------------

# --- airport_fee_merged ---
df_clean.loc[
    (df_clean["airport_fee_merged"].isna()) & (~df_clean["PULocationID"].isin([132, 138])),
    "airport_fee_merged"
] = 0

# --- passenger_count ---
# Imputer NaN par la médiane selon PULocationID
df_clean["passenger_count"] = df_clean["passenger_count"].fillna(
    df_clean.groupby("PULocationID")["passenger_count"].transform("median")
)

# --- RatecodeID ---
# Imputer NaN par le mode selon PULocationID
df_clean["RatecodeID"] = df_clean["RatecodeID"].fillna(
    df_clean.groupby("PULocationID")["RatecodeID"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else 1)
)

# --- store_and_fwd_flag ---
df_clean["store_and_fwd_flag"] = df_clean["store_and_fwd_flag"].fillna("N")

# --- Montants annexes ---
money_cols = ["congestion_surcharge", "airport_fee_merged"]
df_clean[money_cols] = df_clean[money_cols].fillna(0)

# --- Vérification finale ---
print("Nombre de NaN par colonne après nettoyage :")
print(df_clean.isnull().sum())

# Etape 3 : Etude statistique

###1️⃣ PRIX MOYEN D’UNE COURSE (fare_amount)

In [0]:
# sans outliers
# Variable étudiée
fare = df_clean["fare_amount"]

# Taille de l'échantillon
n = len(fare)

# Moyenne et écart-type
mean_fare = fare.mean()
std_fare = fare.std(ddof=1)  # écart-type échantillonnal

# Intervalle de confiance à 95 % (Student)
confidence_level = 0.85
alpha = 1 - confidence_level

ci_low, ci_high = stats.t.interval(
    confidence_level,
    df=n - 1,
    loc=mean_fare,
    scale=std_fare / np.sqrt(n)
)

print("=== Prix moyen d'une course ===")
print(f"Moyenne échantillon : {mean_fare:.2f}")
print(f"IC 95% : [{ci_low:.2f}, {ci_high:.2f}]")
print(f"Taille échantillon : {n}")

In [0]:
# avec outliers
prix_moyen_course = df["fare_amount"].mean()
print(prix_moyen_course)
std  = df["fare_amount"].std()
n= len(df)
Standard_Error = std/ (n ** 0.5)

from scipy import stats
ci_low, ci_high = stats.t.interval(
    0.95,             
    df=n-1,            
    loc=prix_moyen_course,        
    scale=Standard_Error           
)

print("Mean estimate:", prix_moyen_course)
print("95% CI:", ci_low, "-", ci_high)

### 🚩Interpretation :
- Le prix moyen d’une course a été estimé à partir de l’échantillon nettoyé.
La moyenne observée est de 14.89 avec un intervalle de confiance à 95 % égal à
`[14.86 ; 14.91]`. La taille importante de l’échantillon permet d’obtenir une
estimation très précise, comme l’indique l’intervalle de confiance étroit.
__
- La comparaison avec l’échantillon avant nettoyage montre une moyenne plus élevée,
ce qui s’explique par la présence de valeurs aberrantes (courses très chères)
qui influencent fortement la moyenne. Le nettoyage des données permet donc
d’obtenir une estimation plus représentative des courses standards.

###2️⃣ DISTANCE MOYENNE D’UNE COURSE (trip_distance)

In [0]:
# Calcul avec tous les points (outliers inclus)
distance = df["trip_distance"]

n = len(distance)
mean_distance = distance.mean()
std_distance = distance.std(ddof=1)

ci_low, ci_high = stats.t.interval(
    0.95,
    df=n - 1,
    loc=mean_distance,
    scale=std_distance / np.sqrt(n)
)

print("=== Distance moyenne (avec outliers) ===")
print(f"Moyenne échantillon : {mean_distance:.2f}")
print(f"IC 95% : [{ci_low:.2f}, {ci_high:.2f}]")
print(f"Taille échantillon : {n}")

In [0]:
# Calcul sans outliers évidents (distance = 0)
distance_no_outliers = df_clean[~df_clean["flag_distance_zero"]]["trip_distance"]

n_no = len(distance_no_outliers)
mean_distance_no = distance_no_outliers.mean()
std_distance_no = distance_no_outliers.std(ddof=1)

ci_low_no, ci_high_no = stats.t.interval(
    0.95,
    df=n_no - 1,
    loc=mean_distance_no,
    scale=std_distance_no / np.sqrt(n_no)
)

print("\n=== Distance moyenne (sans distances nulles) ===")
print(f"Moyenne échantillon : {mean_distance_no:.2f}")
print(f"IC 95% : [{ci_low_no:.2f}, {ci_high_no:.2f}]")
print(f"Taille échantillon : {n_no}")

### 🚩Interpretation :
- La distance moyenne d’une course a d’abord été estimée sur l’échantillon brut,
incluant les valeurs aberrantes. La moyenne obtenue (5.67 miles) apparaît élevée
et s’accompagne d’un intervalle de confiance large, ce qui indique une forte
variabilité due à la présence de distances nulles et de courses exceptionnellement longues.

- Après nettoyage des données et exclusion des distances nulles, la distance moyenne
estimée est de 2.77 miles avec un intervalle de confiance à 95 % égal à
[2.43 ; 3.12]. Cette valeur est plus cohérente avec des trajets urbains standards.
Cela montre que la variable `trip_distance` est fortement influencée par les outliers
et que le nettoyage des données est indispensable pour obtenir une estimation fiable.


###3️⃣ DURÉE MOYENNE DES COURSES (à partir de pickup / dropoff)

In [0]:
# Durée moyenne (avec durées extrêmes)
duration = df["trip_duration_min"]

n = len(duration)
mean_duration = duration.mean()
std_duration = duration.std(ddof=1)

ci_low, ci_high = stats.t.interval(
    0.95,
    df=n - 1,
    loc=mean_duration,
    scale=std_duration / np.sqrt(n)
)

print("=== Durée moyenne des courses (avec outliers) ===")
print(f"Durée moyenne : {mean_duration:.2f} minutes")
print(f"IC 95% : [{ci_low:.2f}, {ci_high:.2f}]")
print(f"Taille échantillon : {n}")

In [0]:
# Durée moyenne sans durées extrêmes
duration_no_outliers = df_clean[~df_clean["flag_duration_extreme"]]["trip_duration_min"]

n_no = len(duration_no_outliers)
mean_duration_no = duration_no_outliers.mean()
std_duration_no = duration_no_outliers.std(ddof=1)

ci_low_no, ci_high_no = stats.t.interval(
    0.95,
    df=n_no - 1,
    loc=mean_duration_no,
    scale=std_duration_no / np.sqrt(n_no)
)

print("\n=== Durée moyenne des courses (sans outliers extrêmes) ===")
print(f"Durée moyenne : {mean_duration_no:.2f} minutes")
print(f"IC 95% : [{ci_low_no:.2f}, {ci_high_no:.2f}]")
print(f"Taille échantillon : {n_no}")

### 🚩 Interpretation :
- La durée moyenne des courses a été calculée à partir de l’échantillon nettoyé.
La moyenne observée est de `14.03 minutes`, avec un intervalle de confiance à 95 % de `[14.01 ; 14.05]`.

- La comparaison avec l’échantillon brut (`moyenne 26.08 min`, `IC [9.10 ; 43.06]`) montre l’impact des valeurs extrêmes : certaines courses très longues influencent fortement la moyenne et élargissent l’IC.
Le nettoyage des données permet donc d’obtenir une estimation plus représentative de la durée réelle des courses standards.

###4️⃣ PROPORTION DES COURSES AVEC TIP > 0

In [0]:
# Proportion avec outliers
# laisser les courses avec tip > 0
tip_out_positive = df["tip_amount"] > 0

In [0]:
# Proportion observée dans l’échantillon
n = len(df)
k = tip_out_positive.sum()   # nombre de courses avec tip > 0

p_hat = k / n

print("=== Proportion des courses avec tip > 0 ===")
print(f"Proportion observée : {p_hat:.4f}")
print(f"Nombre total de courses : {n}")
print(f"Nombre avec tip > 0 : {k}")

In [0]:
# Intervalle de confiance à 95 % (méthode normale)
z = stats.norm.ppf(0.975)  # quantile 97.5%

se = np.sqrt((p_hat * (1 - p_hat)) / n)

ci_low = p_hat - z * se
ci_high = p_hat + z * se

print("\nIC 95 % pour la proportion :")
print(f"[{ci_low:.4f}, {ci_high:.4f}]")

In [0]:
# proportion sans outliers tip :
tip_positive = df_clean["tip_amount"] > 0 # laisser les courses avec tip > 0

In [0]:
# Proportion observée dans l’échantillon
n1 = len(df_clean)
k1 = tip_positive.sum()   # nombre de courses avec tip > 0

p_hat1 = k1 / n1

print("=== Proportion des courses avec tip > 0 ===")
print(f"Proportion observée : {p_hat1:.4f}")
print(f"Nombre total de courses : {n1}")
print(f"Nombre avec tip > 0 : {k1}")

In [0]:
# Intervalle de confiance à 95 % (méthode normale)
z1 = stats.norm.ppf(0.975)  # quantile 97.5%

se1 = np.sqrt((p_hat1 * (1 - p_hat1)) / n)

ci_low1 = p_hat1 - z1 * se
ci_high1 = p_hat1 + z1 * se

print("\nIC 95 % pour la proportion :")
print(f"[{ci_low1:.4f}, {ci_high1:.4f}]")

### 🚩 Interpretation :
- La proportion des courses avec un pourboire supérieur à zéro a été estimée à 0.780 (78 %) dans l’échantillon nettoyé.

- L’intervalle de confiance à 95 % est très étroit : [0.7791 ; 0.7809], reflétant la taille importante de l’échantillon.

- La comparaison avec les données sans valeurs extrêmes montre que les outliers ont un impact négligeable sur cette proportion (0.7798).

- Ainsi, environ 78 % des courses donnent lieu à un pourboire, ce qui est une information robuste pour l’entreprise.

###5️⃣ DISTRIBUTION DES COURSES
Par `heure` / `jour` / `semaine` – Identification des heures de pointe

In [0]:
# Extraction des composantes temporelles
df_clean["pickup_hour"] = df_clean["tpep_pickup_datetime"].dt.hour
df_clean["pickup_dayofweek"] = df_clean["tpep_pickup_datetime"].dt.dayofweek
df_clean["pickup_week"] = df_clean["tpep_pickup_datetime"].dt.isocalendar().week

In [0]:
# Distribution des courses par heure
hourly_counts = df_clean["pickup_hour"].value_counts().sort_index()

hourly_prop = hourly_counts / hourly_counts.sum()

In [0]:
# Intervalle de confiance (proportion horaire)
hour = 18
k = hourly_counts.loc[hour]
n = len(df_clean)

p_hat = k / n
z = stats.norm.ppf(0.975)
se = np.sqrt((p_hat * (1 - p_hat)) / n)

ci_low = p_hat - z * se
ci_high = p_hat + z * se

print(f"Heure {hour}h :")
print(f"Proportion : {p_hat:.4f}")
print(f"IC 95% : [{ci_low:.4f}, {ci_high:.4f}]")

In [0]:
# Distribution par jour de la semaine
day_counts = df_clean["pickup_dayofweek"].value_counts().sort_index()

day_counts

In [0]:
# Distribution hebdomadaire (saisonnalité légère)
weekly_counts = df_clean["pickup_week"].value_counts().sort_index()

plt.figure(figsize=(10, 5))
hourly_counts.plot(kind="bar")
plt.title("Distribution des courses par heure")
plt.xlabel("Heure de pickup")
plt.ylabel("Nombre de courses")
plt.show()

###🚩 Interpretation :
- L’analyse temporelle des courses montre une forte dépendance à l’heure et au jour de la semaine.

- Les heures de pointe se situent principalement entre 16h et 19h, avec un maximum à 18h représentant environ 7.2 % des courses.

- L’intervalle de confiance à 95 % pour cette proportion est très étroit (`[0.0716 ; 0.0728]`), ce qui reflète la grande taille de l’échantillon et la stabilité de cette estimation.

- La distribution par jour de la semaine montre une activité plus élevée du mardi au vendredi, et une baisse le dimanche.

==> Ces résultats indiquent que l’échantillon est temporellement représentatif du comportement global de mobilité urbaine.

###6️⃣ COMPARAISON DES FARES SELON LES ZONES GÉOGRAPHIQUES
`pickup` / `dropoff boroughs`

In [0]:
# Prix moyen par borough (pickup)
fare_by_borough_pickup = (
    df_clean
    .groupby("PULocationID")["fare_amount"]
    .agg(["count", "mean", "std"])
    .reset_index()
)

fare_by_borough_pickup.head()

In [0]:
# Intervalle de confiance à 95 % par borough
def confidence_interval(mean, std, n, alpha=0.05):
    if n <= 1:
        return (np.nan, np.nan)
    t = stats.t.ppf(1 - alpha/2, df=n-1)
    margin = t * std / np.sqrt(n)
    return mean - margin, mean + margin


fare_by_borough_pickup["ci_low"], fare_by_borough_pickup["ci_high"] = zip(*fare_by_borough_pickup.apply(
    lambda row: confidence_interval(row["mean"], row["std"], row["count"]),
    axis=1
))

In [0]:
# Resultat
fare_by_borough_pickup.sort_values("mean", ascending=False)

In [0]:
# Prix moyen par borough (dropoff)
fare_by_borough_dropoff = (
    df_clean
    .groupby("DOLocationID")["fare_amount"]
    .agg(["count", "mean", "std"])
    .reset_index()
)

fare_by_borough_dropoff.head()

In [0]:
# Intervalle de confiance à 95 % par borough
fare_by_borough_dropoff["ci_low"], fare_by_borough_dropoff["ci_high"] = zip(*fare_by_borough_dropoff.apply(
    lambda row: confidence_interval(row["mean"], row["std"], row["count"]),
    axis=1
))

In [0]:
# Resultat
fare_by_borough_dropoff.sort_values("mean", ascending=False)

In [0]:
# Préparer les données (zones bien représentées) : on filtre les zones avec trop peu de courses, sinon les graphes sont trompeurs.
MIN_COUNT = 30
pickup_plot = fare_by_borough_pickup[fare_by_borough_pickup["count"] >= MIN_COUNT]
dropoff_plot = fare_by_borough_dropoff[fare_by_borough_dropoff["count"] >= MIN_COUNT]

In [0]:
# Visualisation : prix moyen par Pickup zone
# Top 20 zones pickup les plus chères
top_pickup = pickup_plot.sort_values("mean", ascending=False).head(20)

plt.figure(figsize=(10, 6))
plt.barh(
    top_pickup["PULocationID"].astype(str),
    top_pickup["mean"],
    xerr=top_pickup["mean"] - top_pickup["ci_low"]
)

plt.xlabel("Fare moyen ($)")
plt.ylabel("PULocationID")
plt.title("Top 20 zones Pickup – Fare moyen (IC 95%)")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [0]:
# Visualisation : prix moyen par Dropoff zone
# Top 20 zones dropoff les plus chères
top_dropoff = dropoff_plot.sort_values("mean", ascending=False).head(20)

plt.figure(figsize=(10, 6))
plt.barh(
    top_dropoff["DOLocationID"].astype(str),
    top_dropoff["mean"],
    xerr=top_dropoff["mean"] - top_dropoff["ci_low"]
)

plt.xlabel("Fare moyen ($)")
plt.ylabel("DOLocationID")
plt.title("Top 20 zones Dropoff – Fare moyen (IC 95%)")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [0]:
# Comparaison visuelle Pickup vs Dropoff (distribution globale)
# Boxplot des fares
plt.figure(figsize=(8, 6))

plt.boxplot(
    [
        df_clean["fare_amount"],
        df_clean.groupby("PULocationID")["fare_amount"].mean(),
        df_clean.groupby("DOLocationID")["fare_amount"].mean()
    ],
    labels=["Toutes courses", "Pickup zones", "Dropoff zones"],
    showfliers=False
)

plt.ylabel("Fare amount ($)")
plt.title("Distribution des fares – global vs spatial")
plt.tight_layout()
plt.show()


###🚩 Interpretation :
- L’analyse spatiale des fares montre une forte hétérogénéité selon les zones de pickup et de dropoff.

- Certaines zones présentent des prix moyens significativement plus élevés, notamment celles associées à des trajets longs ou à des zones périphériques.

- Afin d’éviter des estimations biaisées, seules les zones avec un nombre suffisant de courses (≥ 30) ont été retenues.

- Les intervalles de confiance à 95 % permettent d’évaluer la fiabilité des estimations par zone.

- La comparaison globale montre que l’agrégation spatiale révèle une structure des prix qui est masquée par la moyenne globale.

- ###7️⃣ Ratio tip / fare moyen par type de paiement (cash vs card)

In [0]:
# Préparer les données (logique métier)
# Le tip n’est pertinent que si fare > 0

df_tip = df_clean[df_clean["fare_amount"] > 0].copy()

# Ratio tip / fare
df_tip["tip_fare_ratio"] = df_tip["tip_amount"] / df_tip["fare_amount"]

In [0]:
# Séparer cash vs card
card = df_tip[df_tip["payment_type"] == 1]["tip_fare_ratio"]
cash = df_tip[df_tip["payment_type"] == 2]["tip_fare_ratio"]

print("Nombre de courses :")
print(f"Card : {len(card)}")
print(f"Cash : {len(cash)}")

In [0]:
# onction statistique (scipy – IC 95%)
def mean_ci(data, confidence=0.95):
    data = data.dropna()
    n = len(data)
    mean = data.mean()
    std = data.std(ddof=1)
    
    ci_low, ci_high = stats.t.interval(
        confidence,
        df=n-1,
        loc=mean,
        scale=std / np.sqrt(n)
    )
    
    return mean, std, ci_low, ci_high
    
card_stats = mean_ci(card)
cash_stats = mean_ci(cash)

In [0]:
# Resultat
print("=== RATIO TIP / FARE PAR TYPE DE PAIEMENT ===\n")

print("💳 Carte bancaire")
print(f"Moyenne : {card_stats[0]:.4f}")
print(f"IC 95% : [{card_stats[2]:.4f}, {card_stats[3]:.4f}]\n")

print("💵 Espèces")
print(f"Moyenne : {cash_stats[0]:.4f}")
print(f"IC 95% : [{cash_stats[2]:.4f}, {cash_stats[3]:.4f}]")


In [0]:
# Visualisation
labels = ["Card", "Cash"]
means = [card_stats[0], cash_stats[0]]
errors = [
    card_stats[0] - card_stats[2],
    cash_stats[0] - cash_stats[2]
]

plt.figure(figsize=(10,5))
plt.bar(labels, means, yerr=errors)
plt.ylabel("Ratio tip / fare")
plt.title("Comparaison du ratio tip/fare selon le type de paiement")
plt.tight_layout()
plt.show()

### 🚩 Interpretation :
- Le ratio moyen tip/fare est estimé à 25.1 % pour les paiements par carte bancaire, avec un intervalle de confiance à 95 % très étroit, indiquant une estimation fiable.
- Pour les paiements en espèces, le ratio est nul, ce qui s’explique par le fait que les pourboires en cash ne sont pas enregistrés dans la base de données TLC.
- Cette différence ne reflète donc pas un comportement client, mais une limite structurelle des données disponibles.